In [1]:
%pip install onnxruntime

%pip install -U scikit-image

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade albumentations

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import random
import shutil
import albumentations as A
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed
from skimage.io import imread
from threading import Lock

# Path ke folder data
images_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7"
labels_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7xml"
output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\\temp"
sample_output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\sample_output"

train_val_split = 0.8  # Rasio data train/validation

# Cek apakah path gambar dan label ada
if not os.path.exists(images_dir):
    raise FileNotFoundError(f"Path gambar tidak ditemukan: {images_dir}")
if not os.path.exists(labels_dir):
    raise FileNotFoundError(f"Path label tidak ditemukan: {labels_dir}")

# Remove output directory if it exists and create fresh output folders
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

if os.path.exists(sample_output_dir):
    shutil.rmtree(sample_output_dir)

# Buat ulang folder output yang fresh
os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
os.makedirs(sample_output_dir, exist_ok=True)

# Daftar semua gambar dan annotations
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
label_files = [f.replace('.png', '.xml') for f in image_files]

# Shuffle data for randomness
data = list(zip(image_files, label_files))
random.shuffle(data)

# Split data into training and validation sets
split_index = int(train_val_split * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

class_mapping = {
    'G001': 0, 'G002': 1, 'G003': 2, 'G004': 3, 'G005': 4, 'G006': 5, 'G007': 6, 'G008': 7, 'G009': 8, 'G010': 9,
    'G011': 10, 'G012': 11, 'G013': 12, 'G014': 13, 'G015': 14, 'G016': 15, 'G017': 16, 'G018': 17, 'G019': 18, 'G020': 19,
    'G021': 20, 'G022': 21, 'G023': 22, 'G024': 23, 'G025': 24, 'G026': 25, 'G027': 26, 'G028': 27, 'G029': 28, 'G030': 29,
    'G031': 30, 'G032': 31, 'G033': 32, 'G034': 33, 'G035': 34, 'G036': 35, 'G037': 36, 'G038': 37, 'G039': 38, 'G040': 39,
    'G041': 40, 'G042': 41, 'G043': 42, 'G044': 43, 'G045': 44, 'G046': 45, 'G047': 46, 'G048': 47, 'G049': 48, 'G050': 49,
    'G051': 50, 'G052': 51, 'G053': 52
}

# Augmentasi gambar dan bounding box tanpa mirroring, mengisi area kosong dengan putih
augmentor = A.Compose([
    A.Rotate(limit=30, p=0.3),
    A.RandomBrightnessContrast(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Resize(512, 512, p=1.0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

sample_counter_lock = Lock()

def draw_bboxes(image, bboxes, class_labels):
    """Gambar bounding box pada gambar."""
    for bbox, class_id in zip(bboxes, class_labels):
        x_center, y_center, width, height = bbox
        img_h, img_w = image.shape[:2]
        xmin = int((x_center - width / 2) * img_w)
        ymin = int((y_center - height / 2) * img_h)
        xmax = int((x_center + width / 2) * img_w)
        ymax = int((y_center + height / 2) * img_h)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(image, str(class_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

def convert_xml_to_txt(xml_path, txt_output_path, augment=False, img=None):
    """Fungsi ini akan mengonversi format XML ke format YOLO (txt)."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    bboxes = []
    class_labels = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = class_mapping.get(class_name, -1)
        if class_id == -1:
            continue
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        x_center = (xmin + xmax) / 2.0 / width
        y_center = (ymin + ymax) / 2.0 / height
        box_width = (xmax - xmin) / width
        box_height = (ymax - ymin) / height
        bboxes.append([x_center, y_center, box_width, box_height])
        class_labels.append(class_id)

    with open(txt_output_path, 'w') as txt_file:
        for bbox, label in zip(bboxes, class_labels):
            txt_file.write(f"{label} {' '.join(map(str, bbox))}\n")

    return img, bboxes, class_labels

def apply_morphology(img, kernel_size=(10, 10), blur_kernel=(5, 5)):
    """Terapkan deteksi tepi Canny dengan beberapa langkah pra-pemrosesan."""
    # Jika gambar berwarna, konversi ke grayscale
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    
    # Image preprocessing - Gaussian Blur untuk mengurangi noise awal
    blurred = cv2.GaussianBlur(gray, blur_kernel, 0)
    
    # Gradient calculation - menggunakan deteksi tepi Canny
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)
    
    # Non-max suppression dilakukan secara internal oleh fungsi Canny
    # Double thresholding dan edge tracking by hysteresis juga dilakukan oleh fungsi Canny
    
    return edges

def process_data(img_file, lbl_file, img_dest_folder, lbl_dest_folder, augment=False, sample_output_count=15):
    img_src_path = os.path.join(images_dir, img_file)
    lbl_src_path = os.path.join(labels_dir, lbl_file)

    # Baca gambar dan konversi ke grayscale
    img = imread(img_src_path)
    img = apply_morphology(img)
    
    if augment:
        img_dest_path = os.path.join(img_dest_folder, img_file.replace('.png', '_aug.png'))
        lbl_dest_path = os.path.join(lbl_dest_folder, lbl_file.replace('.xml', '_aug.txt'))
        augmented_img, augmented_bboxes, class_labels = convert_xml_to_txt(lbl_src_path, lbl_dest_path, augment=True, img=img)

        augmented = augmentor(image=augmented_img, bboxes=augmented_bboxes, class_labels=class_labels)
        augmented_img = augmented['image']
        augmented_bboxes = augmented['bboxes']

        # Draw bounding boxes on augmented image and save to sample output
        with sample_counter_lock:
            if sample_output_count > 0:
                sample_img = augmented_img.copy()
                draw_bboxes(sample_img, augmented_bboxes, class_labels)
                cv2.imwrite(os.path.join(sample_output_dir, f"{img_file.replace('.png', '_aug_sample.png')}") , sample_img)
                sample_output_count -= 1

        # Save the augmented image in the train directory
        cv2.imwrite(img_dest_path, augmented_img)
        print(f"Augmented Image saved to: {img_dest_path}")
    else:
        img_dest_path = os.path.join(img_dest_folder, img_file)
        lbl_dest_path = os.path.join(lbl_dest_folder, lbl_file.replace('.xml', '.txt'))
        img = apply_morphology(img)
        cv2.imwrite(img_dest_path, img)
        convert_xml_to_txt(lbl_src_path, lbl_dest_path)
        print(f"Original Image saved to: {img_dest_path}")

def copy_and_convert_data(data, img_dest_folder, lbl_dest_folder, augment=False, sample_output_count=30):
    with ThreadPoolExecutor() as executor:
        futures = []
        for img_file, lbl_file in data:
            futures.append(executor.submit(process_data, img_file, lbl_file, img_dest_folder, lbl_dest_folder, augment, sample_output_count))
        for future in as_completed(futures):
            future.result()

copy_and_convert_data(train_data, os.path.join(output_dir, 'images/train'), os.path.join(output_dir, 'labels/train'), augment=True)
copy_and_convert_data(val_data, os.path.join(output_dir, 'images/val'), os.path.join(output_dir, 'labels/val'))

print("Data berhasil dipisah, dikonversi, dan diaugmentasi ke format YOLO! Contoh gambar disimpan di folder sample_output.")

Augmented Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G022-122-0400_aug.png
Augmented Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G025-029-0400_aug.png
Augmented Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G036-047-0400_aug.png
Augmented Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G017-058-0400_aug.png
Augmented Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yol

In [4]:
import os
import numpy as np
from collections import Counter

# Path ke folder label YOLO
label_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\\runs\\train"

# Daftar nama kelas sesuai urutan dalam konfigurasi `names`
class_names = [
    'Oscillatoria', 'Ankistrodesmus', 'Microcystis', 'Gomphonema', 
    'Sphaerocystis', 'Cosmarium', 'Cocconeis', 'Tribonema', 
    'Chlorella', 'Tetraedron', 'Ankistrodesmus', 'Brachionus', 
    'Chaenea', 'Pediastrum', 'Spirulina', 'Actinastrum', 
    'Navicula', 'Scenedesmus', 'Golenkinia', 'Pinnularia', 
    'Staurastrum', 'Phormidium', 'Fragilaria', 'Anabaenopsis', 
    'Coelosphaerium', 'Crucigenia', 'Achnanthes', 'Synedra', 
    'Ceratium', 'Pompholyx', 'Merismopedia', 'Spirogyra', 
    'Coelastrum', 'Raphidiopsis', 'Gomphosphaeria', 'Euglena', 
    'Euclanis', 'Keratella', 'diversicornis', 'Surirella', 
    'Characium'
]

# Fungsi menghitung class weights
def calculate_class_weights(label_dir, class_names):
    class_counts = Counter()
    
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(label_dir, label_file), 'r') as f:
                for line in f:
                    try:
                        class_id = int(float(line.split()[0]))  # Mengonversi ke float dahulu
                        class_counts[class_id] += 1
                    except ValueError as e:
                        print(f"Error parsing line '{line.strip()}' in file '{label_file}': {e}")
                        continue

    # Hitung bobot berbasis inverse frequency
    total_instances = sum(class_counts.values())
    class_weights = {class_id: total_instances / count for class_id, count in class_counts.items()}
    
    # Normalisasi bobot agar proporsional
    max_weight = max(class_weights.values(), default=1.0)
    normalized_weights = {class_id: weight / max_weight for class_id, weight in class_weights.items()}
    
    # Memastikan urutan sesuai `class_names`
    weights_list = [normalized_weights.get(i, 1.0) for i in range(len(class_names))]
    
    return weights_list

# Hitung class weights dan cetak
class_weights = calculate_class_weights(label_dir, class_names)
print("Class Weights sesuai urutan kelas:", class_weights)


Class Weights sesuai urutan kelas: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [4]:
from ultralytics import YOLO

name = 'it2'
def main():
    # Inisialisasi model YOLOv8
    model = YOLO('yolov8n.pt')  # Pre-trained model weights untuk YOLOv8
    
    # Tambahkan `class_weights` sebagai parameter dinamis
    model.train(
        data='../data.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name=name,
        workers=12,
        project=f'runs/train/{name}',
        device=0,
        label_smoothing=0.1,
        weight_decay=0.0005,  # Regularisasi weight decay
    )

    # Evaluasi model setelah pelatihan
    model.val()

    # Export model ke format lain (misal: TorchScript, ONNX, CoreML, TensorRT)
    model.export(format='onnx')  # atau format lain yang diinginkan

if __name__ == '__main__':
    main()

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 
New https://pypi.org/project/ultralytics/8.3.29 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.21  Python-3.9.20 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=../data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=12, project=runs/train/it2, name=it211, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, a

train: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\train... 1892 images, 21 backgrounds, 0 corrupt: 100%|██████████| 1892/1892 [00:04<00:00, 453.25it/s]


train: New cache created: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val... 473 images, 5 backgrounds, 0 corrupt: 100%|██████████| 473/473 [00:01<00:00, 452.78it/s]

val: New cache created: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val.cache


...

                   all        473       1082      0.416       0.17      0.185      0.138

100 epochs completed in 0.477 hours.
Optimizer stripped from runs\train\it2\it211\weights\last.pt, 6.3MB
Optimizer stripped from runs\train\it2\it211\weights\best.pt, 6.3MB

Validating runs\train\it2\it211\weights\best.pt...
Ultralytics 8.3.21  Python-3.9.20 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 168 layers, 3,013,643 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.54it/s]


                   all        473       1082      0.425      0.225      0.222      0.163
          Oscillatoria         18         43      0.214      0.093      0.146       0.09
        Ankistrodesmus         10         10          0          0    0.00497    0.00321
           Microcystis        106        199      0.416      0.161      0.161      0.108
            Gomphonema         18         19      0.579      0.508      0.623      0.489
         Sphaerocystis          4          4      0.222        0.5      0.229      0.116
             Cosmarium          6          6          0          0      0.162      0.147
             Cocconeis          4          4          1          0     0.0323     0.0258
             Tribonema         12         15      0.151        0.4      0.179      0.126
             Chlorella         25         28      0.579      0.344       0.37      0.277
            Tetraedron          4          8          1          0      0.011    0.00307
        Ankistrodesmu

val: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val.cache... 473 images, 5 backgrounds, 0 corrupt: 100%|██████████| 473/473 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:03<00:00,  9.43it/s]


                   all        473       1082      0.423       0.22      0.221      0.162
          Oscillatoria         18         43      0.214      0.093      0.146     0.0897
        Ankistrodesmus         10         10          0          0    0.00499    0.00323
           Microcystis        106        199      0.415      0.161      0.161      0.108
            Gomphonema         18         19      0.579      0.506      0.623       0.49
         Sphaerocystis          4          4      0.166      0.356      0.229      0.121
             Cosmarium          6          6          0          0      0.162      0.148
             Cocconeis          4          4          1          0     0.0323     0.0258
             Tribonema         12         15      0.151        0.4      0.175      0.124
             Chlorella         25         28      0.575      0.339      0.369      0.273
            Tetraedron          4          8          1          0     0.0104    0.00294
        Ankistrodesmu

In [ ]:
import shutil
import os
import datetime

source_dir = os.getcwd()
destination_dir = f'runs/train/{name}'

os.makedirs(destination_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    file_path = os.path.join(source_dir, filename)
    if os.path.isfile(file_path):
        shutil.move(file_path, destination_dir)

True
